In [1]:
from main import compute_inventory

filename = "data/exposure_conditions_divertor/WEST/Hao/P1.0e21_wall_data.mat"
# filename = "data/exposure_conditions_divertor/WEST/Julien/WPN54696-1.5MW-FESTIM_inputs.csv"
res = compute_inventory(filename)